In [ ]:
# BloomFilter.py
#libraries
from BitVector import BitVector
import math
import mmh3
from hashlib import sha256
import hmac,hashlib,io
from hashlib import pbkdf2_hmac
from bitarray import bitarray

class BloomFilter:
    '''
    Bloom Filter Class
    '''
    def __init__(self, expected_num=0, prob=0):
        '''
        expected_num: Number of words stored in the bloom filter
        prob: False Positives probability
        '''
        self.length=self.calc_length(expected_num, prob)
        self.prob=prob
        self.hash_num=self.optimum_hashes(self.length ,expected_num)
        self.C=self.compute_cr(self.length, expected_num)
        try:
            self.bit_vector=int(self.length)*bitarray('0')
        except ValueError as exp:
            print(f"An exception of the type {exp} has occured")
    
    @classmethod
    def calc_length(self, n1, n2): 
        '''
        calculating size of array
        '''
        n = (-(n1*math.log(n2))/(math.log(2))**2) 
        return n
        
    # Question 8
    @classmethod
    def compute_cr(self,m,expected_num):
        '''
        calculating the compression rate (CR)
        '''
        return m/expected_num
    
    @classmethod
    def optimum_hashes(self, n1, n2):
        '''
        calculating number of hash functions to use
        '''
        n=((n1/n2)*math.log(2))
        return int(n)
    
    def insert(self, data):
        '''
        adding item to the bloom filter
        '''
        if type(data) is int:
            data=str(data)
        if self.verify(data)==False:
            position_pointer=0
            for i in range(self.hash_num):
                if i%2==0:
                    position_pointer=int(mmh3.hash128(data, i,False)%int(self.length))
                    self.bit_vector[position_pointer]=True
                else:
                    position_pointer=int(pbkdf2_hmac('sha256', data.encode('utf-8'), str(i).encode('utf-8') * 2, 500_000).hex(),16)%int(self.length)
                    self.bit_vector[position_pointer]=True
            print('The Input definatly did not already exists, hence it has been added')
        else:
            print('The Input probably already exists')

    def verify(self, data):
        '''
        verifying if an element is in the bloom filter
        '''
        if type(data) is int:
            data=str(data)
        position_verify=[None]*int(self.length)
        position_pointer=0
        for i in range(self.hash_num):
            if i%2==0:
                position_pointer=int(mmh3.hash128(data, i,False)%int(self.length))
                
                if self.bit_vector[position_pointer]==1:
                    pass
                else:
                    return False
            else:
                #position_pointer=int(sha256(data.encode('utf-8')).hexdigest()),16)%self.length
                position_pointer=int(pbkdf2_hmac('sha256', data.encode('utf-8'), str(i).encode('utf-8') * 2, 500_000).hex(),16)%int(self.length)
                if self.bit_vector[position_pointer]==1:
                    pass
                else:
                    return False
        return True
    
    def __str__(self):  
        return "The bloom filter in string form is % s" % (self.bit_vector)

#Question.no.7
def fp_bloomfilter(expec, p, adding, max_adds, length_test):
    bloom_filter_test = BloomFilter(expec, p)
    added_items = 0
    
    words_in_BF = [str(i) for i in range(max_adds)]
    words_not_in_BF = [str(i) for i in range(max_adds, max_adds + length_test)]
    
    "adding words and calculating FPR"
    fp_rates = []
    for word in words_in_BF:
        bloom_filter_test.insert(word)
        added_items += 1
        if added_items % adding== 0:
            fp = 0
            for word_test in words_not_in_BF:
                if bloom_filter_test.verify(word_test):
                    fp += 1
            fp_rate = fp / length_test
            fp_rates.append((added_items, fp_rate))
    return fp_rates

#testing
expec = 4 
p = 0.01  
adding = 6  
max_adds = 6  
length_test = 8 

fp_rates = fp_bloomfilter(expec, p, adding, max_adds, length_test)
for count, rate in fp_rates:
    print(f"The number of the added items is {count}, the FP rate is {rate:.4f}")

In [ ]:
#!usr/bin/env/ pyhton
# BF_test.py
import math
import mmh3
from hashlib import sha256
import hmac,hashlib,io
from hashlib import pbkdf2_hmac
from bitarray import bitarray
from BloomFilter import BloomFilter
import numpy as np
import random
from wonderwords import RandomSentence
import time
import matplotlib.pyplot as plt

np.random.seed(0)
random.seed(0)

length=50
probability=0.05
max_size=25

def DNA_data_generate(length):
    lst=[]
    for i in range(length):
        seq_length = random.randint(1,max_size)
        lst.append(''.join(np.random.choice(('C','G','T','A'), seq_length )))
    return lst

def DNA_data_generate2(length,lst_prior):
    lst=[]
    for i in range(length):
        seq_length = random.randint(1,max_size)
        temp=''.join(np.random.choice(('C','G','T','A'), seq_length ))
        if temp not in (lst_prior):
            lst.append(temp)
    return lst


def test(data1,data2):
    
    lst1=data1
    lst2=data2

    bl=BloomFilter(length,probability)
    print('Size of Bloom Filter:', int(bl.length))
    print('Optimal number of Hahes for the Bloom Filter:', int(bl.hash_num))

    for i in range(length):
        bl.insert(lst1[i])
    
    print('Bloom Filter created')

    #test
    count1=0
    count2=0
    false_count=0
    for i in range(len(lst1)):
        if bl.verify(lst1[i])==True:
            count1+=1
    if count1==len(lst1):
        print('Present Data passed')
            
    for i in range(len(lst2)):
        if bl.verify(lst2[i])==False:
            count2+=1
        else:
            print('False Positive: ', lst2[i], bl.bit_vector)
    if count2==len(lst2):
        print('Non-present Data passed')


def gen_sentence_data():
    lst=[]
    for i in range(length):
        s = RandomSentence()
        lst.append(s.sentence())
    
    return lst 

data1=DNA_data_generate(length)
data2=DNA_data_generate2(length, data1)

test(data1,data2)

In [ ]:
# Bloomfilter_test.py
#libraries
from BitVector import BitVector
import math
import mmh3
from hashlib import sha256
import hmac,hashlib,io
from hashlib import pbkdf2_hmac
from bitarray import bitarray
from BloomFilter import BloomFilter
import numpy as np
import random
from wonderwords import RandomSentence
import time
import matplotlib.pyplot as plt

np.random.seed(0)
random.seed(0)

length=20
probability=0.05
max_size=5

def generate_number(length):
    lst=[]
    for i in range(length):
        lst.append(random.randrange(1,100000))
    return lst

def generate_number2(length,lst_prior):
    lst=[]
    for i in range(length):
        temp=(random.randrange(1,100000))
        if temp not in (lst_prior):
            lst.append(temp)
    return lst

num1=(generate_number(length))

num2=(generate_number2(length,num1))

def test(data1,data2):
    lst1=data1
    lst2=data2

    bl=BloomFilter(length,probability)
    print('Size of Bloom Filter:', int(bl.length))
    print('Optimal number of Hahes for the Bloom Filter:', int(bl.hash_num))

    for i in range(length):
        bl.insert(lst1[i])
      
    print('Bloom Filter created')

    #test
    count1=0
    count2=0
    false_count=0
    for i in range(len(lst1)):
        if bl.verify(lst1[i])==True:
           
            count1+=1
    if count1==len(lst1):
        print('Present Data passed')
    
    for i in range(len(lst2)):
        if bl.verify(lst2[i])==False:
            count2+=1
        else:
            print('False Positive: ', lst2[i], bl.bit_vector)
            false_count+=1
    if count2==len(lst2):
        print('Non-present Data passed')
    else:
        print("The False Positives have been printed above and they are: ",false_count)

test(num1,num2)

def gen_sentence_data(length):
    lst=[]
    for i in range(length):
        s = RandomSentence()
        lst.append(s.sentence())
    return lst

In [ ]:
#!usr/bin/env/ pyhton
# insert_test_performance.py
import math
import mmh3
from hashlib import sha256
import hmac,hashlib,io
from hashlib import pbkdf2_hmac
from bitarray import bitarray
from BloomFilter import BloomFilter
import numpy as np
import random
from wonderwords import RandomSentence,RandomWord
import time
import matplotlib.pyplot as plt

def generate_words(n=0):
    words=[]
    for i in range(n):
        r = RandomWord()
        words.append(r.word())
    return set(words)

def main():
    print('insert')
    data=generate_words(1000)
    data_list = list(data)

    sizes = [10, 50,75, 100,200,350, 500, 1_000,5_000]

    samples = [
        random.sample(data_list, k=size) for size in sizes
    ]

    nr_runs = 3
    times = {}
    insert_sample = random.sample(data_list, k=20)
    i=0



    for sample in samples:
        btree = BloomFilter(sizes[i],0.05)
        i+=1
        for word in sample:
            btree.insert(word)
        times[len(sample)] = 0.0
        for _ in range(nr_runs):
            start_time = time.time_ns()
            for word in insert_sample:
                btree.insert(word)
            end_time = time.time_ns()
            times[len(sample)] += end_time - start_time
        times[len(sample)] /= nr_runs*1_000_000.0
        print(times)

    plt.plot(times.keys(), times.values());

    plt.savefig('plotinsert.png')


if __name__ == "__main__":
    main()

In [ ]:
#!usr/bin/env/ pyhton
# verify_test_performance.py
import math
import mmh3
from hashlib import sha256
import hmac,hashlib,io
from hashlib import pbkdf2_hmac
from bitarray import bitarray
from BloomFilter import BloomFilter
import numpy as np
import random
from wonderwords import RandomSentence,RandomWord
import time
import matplotlib.pyplot as plt

def generate_words(n=0):
    '''
    Generating words function
    '''
    words=[]
    for i in range(n):
        r = RandomWord()
        words.append(r.word())
    return set(words)

def main():
    print('verify')
    data=generate_words(1000)
    data_list = list(data)
    # optimal sizes chosen
    sizes = [10, 50,75, 100,200,350, 500, 1_000,5_000]

    samples = [
        random.sample(data_list, k=size) for size in sizes
    ]

    nr_runs = 3
    times = {}
    verify_sample = random.sample(data_list, k=20)
    i=0

    for sample in samples:
        btree = BloomFilter(sizes[i],0.05)
        i+=1
        for word in sample:
            btree.insert(word)
        times[len(sample)] = 0.0
        for _ in range(nr_runs):
            start_time = time.time_ns()
            for word in verify_sample:
                btree.verify(word)
            end_time = time.time_ns()
            times[len(sample)] += end_time - start_time
        times[len(sample)] /= nr_runs*1_000_000.0
        print(times)

# plotting to see the performance
    plt.plot(times.keys(), times.values());
    plt.savefig('plotverify.png')

if __name__ == "__main__":
    main()